# Pytorch Tutorial

### 2. Modules and Custom Models

- Adding predefined modules from ```torch.nn``` to ```torch.nn.Sequential```.
- Creating custom models by inheriting ```torch.nn.Module```.

Some setup.

```torch.nn``` is conventionally imported as ```nn```.  
```torch.nn.functional``` is conventionally imported as ```F```.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## Using ```torch.nn``` modules and ```torch.nn.Sequential```

There are many predefined layers in ```torch.nn```, like conv, linear, pool, ReLU, and more. Also, there are many predefined functions in ```torch.nn.functional```, which overlap a lot with those in ```torch.nn```. The functions are listed in [the Pytorch documentation](https://pytorch.org/docs/stable/nn.html).

The difference is that those defined in ```torch.nn``` are essentially wrappers of functions in ```torch.nn.functional``` plus weight initialization and functions such as ```train()```, ```eval()```, or ```parameters()```. Functionals only compute the bare computation of the layer.

Here we instantiate a simple two dimensional convolution layer and run it.

In [ ]:
X = torch.randn((64, 3, 23, 23))

In [ ]:
two_dimensional_conv = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=11, stride=4, padding=2)
two_dimensional_conv(X)

Now we run the same convolution with an all-zero filter. Try to catch the difference between ```torch.nn``` layers and ```torch.nn.functional``` functions.

In [ ]:
zero_conv_result = F.conv2d(input=X, weight=torch.zeros(64, 3, 11, 11), stride=4, padding=2)
zero_conv_result_pool = F.max_pool2d(zero_conv_result, kernel_size=3, stride=2)
zero_conv_result_pool

With ```nn.Sequential```, we can define simple models that do not require much setup.

In [ ]:
simple_model = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2)
)

simple_model(X)

## Creating Custom Models

To create custom models, we define a class that inherits ```torch.nn.Module```. Then, all we need to define is the ```__init__``` function and the ```forward``` function.

Generally speaking, we define the layers and their initialization in ```__init__``` and define their connections in ```forward```.

In [ ]:
class SimpleNet(nn.Module):
    
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2)
        
        self.dropout1 = nn.Dropout(p=0.3)
        self.fc1 = nn.Linear(64*2*2, num_classes)
        
        for m in self.modules():
            if type(m) in [nn.Conv2d, nn.Linear]:
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
                m.bias.data.fill_(0)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = x.view(-1, 64*2*2)
        x = self.dropout1(x)
        x = self.fc1(x)
        return x

In [ ]:
model = SimpleNet()
model(X).shape